In [37]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sqlite3
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import tensorflow as tf
from sklearn.metrics import mean_absolute_error

In [2]:
database = "data.sqlite"

conn = sqlite3.connect(database)

train_df = pd.read_sql("select * from new_table_name", con=conn)
train_df.head()

,Unnamed: 0,year,make,model,final_price,mileage,engine,zipcode,engine string
0,1,1997,Chevrolet,Chevrolet Suburban,"$17,000",67000,5700.0,60069,5.7L Vortec V8
1,2,1990,Porsche,Porsche 964 911,"$225,000",1000,3800.0,84790,3.8-Liter Flat-Six
2,3,2003,Toyota,Toyota Pickup,"$24,750",116000,3400.0,90027,3.4-Liter DOHC V6
3,5,1992,Volkswagen,Volkswagen Golf/Rabbit Cabriolet,"$10,750",100000,1800.0,98208,1.8-Liter Inline-Four
4,6,2008,Toyota,Toyota FJ Cruiser,"$32,500",9000,4000.0,57108,4.0-Liter V6


In [3]:
conn.close()

In [4]:
train_df['final_price']=(train_df['final_price'].replace( '[\$,)]','', regex=True )
               .replace( '[(]','-',   regex=True ).astype(float))

In [5]:
train_df = train_df.drop(['model', 'engine string', 'Unnamed: 0'], axis='columns')
train_df

,year,make,final_price,mileage,engine,zipcode
0,1997,Chevrolet,17000.0,67000,5700.0,60069
1,1990,Porsche,225000.0,1000,3800.0,84790
2,2003,Toyota,24750.0,116000,3400.0,90027
3,1992,Volkswagen,10750.0,100000,1800.0,98208
4,2008,Toyota,32500.0,9000,4000.0,57108
...,...,...,...,...,...,...
7100,1983,Jeep,39962.0,24000,4200.0,78726
7101,1997,Ford,24900.0,75000,7300.0,80214
7102,1972,Honda,2400.0,5000,174.0,98579
7103,2005,Ford,17750.0,22000,3900.0,97303


In [47]:
train_df.value_counts()

year  make           final_price  mileage  engine    zipcode
2009  Mercedes-Benz  28750.0      33000    6200.000  73112      2
1921  Ford           8111.0       4000     2900.499  95818      1
2002  BMW            55499.0      42000    4900.000  30101      1
      Chevrolet      26000.0      56000    5700.000  52240      1
                                  12000    5300.000  53114      1
                                                               ..
1984  Land Rover     19750.0      94000    2250.000  4103       1
      Jeep           35000.0      137000   4200.000  30060      1
                     26000.0      139000   4200.000  23111      1
                     25000.0      70000    4200.000  85741      1
2022  Porsche        307307.0     25000    4000.000  97227      1
Length: 7104, dtype: int64

In [6]:
# Convert categorical data to numeric and separate target feature for training data
train_df1 = pd.get_dummies(train_df)
train_df1.head()

,year,final_price,mileage,engine,zipcode,make_AMC,make_Acura,make_Alfa Romeo,make_Alpine,make_Amphicar,...,make_Saab,make_Shelby,make_Studebaker,make_Subaru,make_Sunbeam,make_Toyota,make_Triumph,make_Volkswagen,make_Volvo,make_Willys
0,1997,17000.0,67000,5700.0,60069,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1990,225000.0,1000,3800.0,84790,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2003,24750.0,116000,3400.0,90027,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1992,10750.0,100000,1800.0,98208,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2008,32500.0,9000,4000.0,57108,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
train_X = train_df1.drop(columns='final_price')
train_y = train_df1['final_price']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
print(f'Model Score: {model.score(X_train_scaled, y_train)}')
print(f'Model Score: {model.score(X_test_scaled, y_test)}')

Model Score: 0.26891112197770317
Model Score: -1.3442333068456289e+21


In [17]:
reg = Lasso(max_iter=10000).fit(X_train_scaled, y_train)
print(f'Train score: {reg.score(X_train_scaled, y_train)}')
print(f'Test Score: {reg.score(X_test_scaled, y_test)}')

0.26891353510597527
0.07096287291192027


In [18]:
reg = Ridge(alpha=100).fit(X_train_scaled, y_train)
print(f'Train score: {reg.score(X_train_scaled, y_train)}')
print(f'Test Score: {reg.score(X_test_scaled, y_test)}')

0.2688029994241623
0.07296367827482575


In [27]:
reg = ElasticNet(alpha=10).fit(X_train_scaled, y_train)
print(f'Train score: {reg.score(X_train_scaled, y_train)}')
print(f'Test Score: {reg.score(X_test_scaled, y_test)}')

Train score: 0.0899854052964646
Test Score: 0.039336424607241405


In [41]:
def test_model(model, data):
    X_train_scaled, X_test_scaled, y_train, y_test = data
    reg = model.fit(X_train_scaled, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train_scaled, y_train)}')
    print(f'Test Score: {reg.score(X_test_scaled, y_test)}\n')
    plt.show()
    y_pred = reg.predict(X_test_scaled)
    print(mean_absolute_error(y_test, y_pred))    

In [42]:
data = [X_train_scaled, X_test_scaled, y_train, y_test]

In [43]:
test_model(KNeighborsRegressor(), data)
test_model(RandomForestRegressor(), data)
test_model(ExtraTreesRegressor(), data)
test_model(AdaBoostRegressor(), data)
test_model(SVR(C=1.0, epsilon=0.2), data)

Model: KNeighborsRegressor
Train score: 0.5522080042712837
Test Score: 0.2103116205771125

22945.97715250422
Model: RandomForestRegressor
Train score: 0.9230673896674091
Test Score: 0.3864988864343004

19026.908846825845
Model: ExtraTreesRegressor
Train score: 0.9999912315165191
Test Score: 0.4032217541068486

18491.891260551492
Model: AdaBoostRegressor
Train score: -4.006480787717998
Test Score: -2.8245515834201362

118286.67111487917
Model: SVR
Train score: -0.08711080651893743
Test Score: -0.07324365935156885

29267.422239691077


In [44]:
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation="sigmoid", input_dim=72))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=25, activation="sigmoid"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               7300      
                                                                 
 dense_7 (Dense)             (None, 25)                2525      
                                                                 
 dense_8 (Dense)             (None, 1)                 26        
                                                                 
Total params: 9,851
Trainable params: 9,851
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="mse", optimizer="rmsprop", metrics=["mae"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=1000)

Epoch 1/1000
167/167 [==============================] - 1s 2ms/step - loss: 5156459520.0000 - mae: 45892.6680
Epoch 2/1000
167/167 [==============================] - 0s 2ms/step - loss: 5156068352.0000 - mae: 45888.4219
Epoch 3/1000
167/167 [==============================] - 0s 2ms/step - loss: 5155681280.0000 - mae: 45884.1758
Epoch 4/1000
167/167 [==============================] - 0s 2ms/step - loss: 5155289088.0000 - mae: 45879.9531
Epoch 5/1000
167/167 [==============================] - 0s 2ms/step - loss: 5154896384.0000 - mae: 45875.7070
Epoch 6/1000
167/167 [==============================] - 0s 2ms/step - loss: 5154507776.0000 - mae: 45871.4609
Epoch 7/1000
167/167 [==============================] - 0s 2ms/step - loss: 5154121728.0000 - mae: 45867.2461
Epoch 8/1000
167/167 [==============================] - 0s 2ms/step - loss: 5153734144.0000 - mae: 45863.0078
Epoch 9/1000
167/167 [==============================] - 0s 2ms/step - loss: 5153338880.0000 - mae: 45858.7500
Epoch 10/1